In [2]:
!pip install yfinance
!pip install pandas_datareader
!pip install plotly

You should consider upgrading via the '/home/hachan/stock-market-analytic/my-venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/hachan/stock-market-analytic/my-venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/hachan/stock-market-analytic/my-venv/bin/python -m pip install --upgrade pip' command.


In [10]:
import numpy as np
import pandas as pd

import yfinance as yf
import pandas_datareader as pdr

import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date
from datetime import datetime

import matplotlib.pyplot as plt

import requests
from io import StringIO

# Question 1
### What's the total sum ($m) of 2023 filings that happened on Fridays?


In [34]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)
ipo_dfs = pd.read_html(StringIO(response.text))
ipos_fill = ipo_dfs[0]
ipos_fill['Filing Date'] = pd.to_datetime(ipos_fill['Filing Date'], format='%b %d, %Y')
ipos_fill['Shares Offered'] = pd.to_numeric(ipos_fill['Shares Offered'], errors='coerce')

In [44]:
def avg_price(price_string):
    price_string = price_string.replace("$", "")
    if price_string == "-":
        return None
    elif '-' in price_string:
        start, end = map(float, price_string.split('-'))
        return (start + end) / 2
    else:
        return float(price_string)

ipos_fill['avg_price'] = ipos_fill['Price Range'].apply(avg_price)
ipos_fill['Shares_offered_value'] = ipos_fill['avg_price'] * ipos_fill['Shares Offered']

In [45]:
ipos_fill.head(100)

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,avg_price,Shares_offered_value
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.0,11250000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.0,60000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.0,50000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN,NaN
...,...,...,...,...,...,...,...
95,2023-10-03,GRDN,"Guardian Pharmacy Services, Inc.",-,NaN,NaN,NaN
96,2023-09-29,KAPA,"Kairos Pharma, Ltd.",$4.00,1550000.0,4.0,6200000.0
97,2023-09-29,VAPA,Valens Pay Global Limited,$5.00 - $6.00,1000000.0,5.5,5500000.0
98,2023-09-28,TKE,OUI Global,$4.00 - $6.00,2500000.0,5.0,12500000.0


In [50]:
filtered_rows = ipos_fill[(ipos_fill['Filing Date'].dt.year == 2023) & (ipos_fill['Filing Date'].dt.dayofweek == 4)]
filtered_rows['Shares_offered_value'].sum() / 1_000_000

285.7

# Question 2
### IPOs "Fixed days hold" strategy

In [68]:
url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(StringIO(response.text))
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [69]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(StringIO(response.text))
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      65 non-null     object
 1   Symbol        65 non-null     object
 2   Company Name  65 non-null     object
 3   IPO Price     65 non-null     object
 4   Current       65 non-null     object
 5   Return        65 non-null     object
dtypes: object(6)
memory usage: 3.2+ KB


In [70]:
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format='%b %d, %Y')
filtered_df = stacked_ipos_df[stacked_ipos_df['IPO Date'] < '2024-03-01']
filtered_df.head(3)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
34,2024-02-27,SMXT,"SolarMax Technology, Inc.",$4.00,$10.69,167.25%
35,2024-02-22,VHAI,Vocodia Holdings Corp,$4.25,$0.14,-96.66%
36,2024-02-21,DYCQ,DT Cloud Acquisition Corporation,$10.00,$10.16,1.60%


In [71]:
filtered_df.count()

IPO Date        185
Symbol          185
Company Name    185
IPO Price       185
Current         185
Return          185
dtype: int64

In [65]:
def retrieve_prices(ticker, start_date, end_date):
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        return stock_data
    except Exception as e:
        print(f"Failed to retrieve data for {ticker}: {e}")
        return None


In [74]:
start_date = filtered_df['IPO Date'].min()
end_date = datetime.today().strftime('%Y-%m-%d')

new_symbol_dict = {'PTHR': 'HOVR'}

ohlc_prices = {}
for ticker in filtered_df['Symbol']:
    new_ticker = new_symbol_dict.get(ticker)
    price_ticker = new_ticker if new_ticker else ticker
    prices = retrieve_prices(price_ticker, start_date, end_date)
    if prices is not None:
        ohlc_prices[ticker] = prices
        
del ohlc_prices['RYZB']
print("Data retrieval complete.")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Data retrieval complete.


In [145]:
data = []
#print(filtered_df[filtered_df['Symbol'] != 'RYZB']['Symbol'].tolist())
for ticker, price_df in ohlc_prices.items():
    price_df['Ticker'] = ticker
    data.append(price_df)
df_ohlcv = pd.concat(data)
for i in range(1, 31):
    df_ohlcv[f'growth_future_{i}d'] = df_ohlcv.groupby('Ticker')['Adj Close'].pct_change(-i)
min_date_indices = df_ohlcv.groupby('Ticker').idxmin()
min_date_rows = df_ohlcv.groupby('Ticker').apply(lambda x: x.loc[x.index.min()], include_groups=False)

In [136]:
quantiles = min_date_indices[[f'growth_future_{i}d' for i in range(1, 31)]].quantile(0.75)
print(quantiles.idxmax())
quantiles = min_date_rows[[f'growth_future_{i}d' for i in range(1, 31)]].quantile(0.75)
print(quantiles.idxmax())
quantiles = df_ohlcv[[f'growth_future_{i}d' for i in range(1, 31)]].quantile(0.75)
print(quantiles.idxmax())

growth_future_6d
growth_future_26d
growth_future_30d


In [153]:
cols = [i for i in df_ohlcv.keys() if i.find('growth')>=0]
stocks_df_growth = df_ohlcv[cols]
stocks_df_growth_desc = stocks_df_growth.describe()
print(stocks_df_growth_desc.loc['75%'].idxmax())

growth_future_30d


# Question 3
### Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?

In [4]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA


In [5]:
def ticker_downloader(tickers, category, start='2013-12-14', end='2023-12-31'):
    data = []
    for ticker in tickers:
        prices = yf.download(ticker, period="max", start=start, end=end, interval="1d")
        prices['Ticker'] = ticker
        prices['Category']= category
        prices['growth_7d'] = prices['Adj Close'] / prices['Adj Close'].shift(7)
        data.append(prices)
    return pd.concat(data)


In [6]:
largest_stocks_df = ticker_downloader(LARGEST_STOCKS, 'Largest')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [7]:
large_stocks_df = ticker_downloader(LARGE_STOCKS, 'Large')
print(large_stocks_df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2019-09-11    34.868782    35.511101    33.207928    34.038357    33.763878   
2019-09-12    34.501743    34.524685    32.978527    33.148285    32.880985   
2019-09-13    33.171223    34.042942    32.804184    33.905304    33.631897   
2019-09-16    33.515324    33.721783    32.340797    32.340797    32.080013   
2019-09-17    32.758305    33.033585    31.574600    31.794825    31.538439   
...                 ...          ...          ...          ...          ...   
2023-12-22  3580.000000  3638.449951  3560.550049  3627.350098  3627.350098   
2023-12-26  3635.000000  3665.000000  3623.449951  3656.699951  3656.699951   
2023-12-27  3668.000000  3695.000000  3645.000000  3689.250000  3689.250000   
2023-12-28  3699.899902  3737.000000  3680.699951  3715.100098  3715.100098   
2023-12-29  3715.100098  3715.100098  3660.250000  3

In [8]:
largest_stocks_df = largest_stocks_df[largest_stocks_df.index >= pd.to_datetime('2014-01-01')]
large_stocks_df = large_stocks_df[large_stocks_df.index >= pd.to_datetime('2014-01-01')]

In [9]:
avg_growth_largest_df = largest_stocks_df.groupby(by=['Date','Category']).growth_7d.mean()
print(avg_growth_largest_df)
avg_growth_large_df = large_stocks_df.groupby(by=['Date','Category']).growth_7d.mean()
print(avg_growth_large_df)

Date        Category
2014-01-01  Largest     1.011797
2014-01-02  Largest     1.006333
2014-01-03  Largest     0.999152
2014-01-06  Largest     0.994203
2014-01-07  Largest     0.992509
                          ...   
2023-12-22  Largest     1.013788
2023-12-26  Largest     1.014900
2023-12-27  Largest     1.014155
2023-12-28  Largest     1.008533
2023-12-29  Largest     1.005337
Name: growth_7d, Length: 2595, dtype: float64
Date        Category
2014-01-01  Large       1.011684
2014-01-02  Large       1.002958
2014-01-03  Large       1.000875
2014-01-06  Large       0.995983
2014-01-07  Large       0.991592
                          ...   
2023-12-22  Large       1.003014
2023-12-26  Large       1.013505
2023-12-27  Large       1.002167
2023-12-28  Large       0.998583
2023-12-29  Large       0.999313
Name: growth_7d, Length: 2595, dtype: float64


In [11]:
#large_stocks_df = large_stocks_df.loc[large_stocks_df.index >= largest_stocks_df.index[0]]
merged_data = pd.merge(avg_growth_largest_df, avg_growth_large_df, on='Date', suffixes=('_largest', '_large'))
print(merged_data)
outperformance_days = (merged_data['growth_7d_large'] > merged_data['growth_7d_largest']).sum()
print(outperformance_days)
total_days = len(merged_data)
print(total_days)
pct = int((outperformance_days / total_days) * 100)
print(pct)

            growth_7d_largest  growth_7d_large
Date                                          
2014-01-01           1.011797         1.011684
2014-01-02           1.006333         1.002958
2014-01-03           0.999152         1.000875
2014-01-06           0.994203         0.995983
2014-01-07           0.992509         0.991592
...                       ...              ...
2023-12-22           1.013788         1.003014
2023-12-26           1.014900         1.013505
2023-12-27           1.014155         1.002167
2023-12-28           1.008533         0.998583
2023-12-29           1.005337         0.999313

[2595 rows x 2 columns]
1231
2595
47


# Question 4
### What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

In [13]:
!pip install pyarrow
!pip install fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.1 MB/s eta 0:00:00m eta 0:00:010:00:01
You should consider upgrading via the '/home/hachan/stock-market-analytic/my-venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.8 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.2 MB/s eta 0:00:0031m21.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 KB 20.6 MB/s eta 0:00:00
You should consider upgrading via the '/home/hachan/stock-market-analytic/my-venv/bin/python -m pip install --upgrade pip' command.


In [16]:
import pyarrow
import fastparquet

file = "stocks_df_combined_trunc_2014_2023.parquet.brotli"
df = pd.read_parquet(file, engine='pyarrow')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80762 entries, 7011 to 5342
Columns: 202 entries, Open to growth_btc_usd_365d
dtypes: datetime64[ns](3), float64(128), int32(64), int64(5), object(2)
memory usage: 105.4+ MB


In [22]:
df.head()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
print(list(df.columns))

['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker', 'Year', 'Month', 'Weekday', 'Date', 'growth_1d', 'growth_3d', 'growth_7d', 'growth_30d', 'growth_90d', 'growth_365d', 'growth_future_5d', 'SMA10', 'SMA20', 'growing_moving_average', 'high_minus_low_relative', 'volatility', 'is_positive_growth_5d_future', 'ticker_type', 'index_x', 'adx', 'adxr', 'apo', 'aroon_1', 'aroon_2', 'aroonosc', 'bop', 'cci', 'cmo', 'dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext', 'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix', 'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo', 'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk', 'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr', 'index_y', 'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod', 'ht_dcphase', 'ht_phasor_inphase', 'ht_phasor_quadrature', 'ht_sine_sine', 'ht_sine_leadsine', 'ht_trendmod', 'avgprice', 'medprice', 'typprice', 'wclprice', 'index', 'cdl2crows', 'cdl3blackrows', 'c

In [52]:
filter_df = df[df['Weekday'] == 4]
filter_df = filter_df[filter_df['cci'] >= 200]
filter_df = filter_df[['Ticker', 'Date', 'Weekday', 'Adj Close_x', 'Adj Close_y', 'growth_future_5d', 'cci']]

In [53]:
filter_df.head()

,Ticker,Date,Weekday,Adj Close_x,Adj Close_y,growth_future_5d,cci
7113,MSFT,2014-05-30,4,34.912762,11.400000,1.013190,203.518798
7181,MSFT,2014-09-05,4,39.395618,12.090000,1.017207,257.013330
7340,MSFT,2015-04-24,4,41.630741,12.290000,1.016503,340.009871
7452,MSFT,2015-10-02,4,40.151123,20.940001,1.033794,206.125828
7467,MSFT,2015-10-23,4,46.583046,14.460000,0.995650,431.107024


In [54]:
filter_df['profit'] = 1000 * (filter_df['growth_future_5d'] - 1)

In [55]:
filter_df['profit'].sum()

1048.2628918177302

In [56]:
filter_df.count()

Ticker              460
Date                460
Weekday             460
Adj Close_x         460
Adj Close_y         459
growth_future_5d    460
cci                 460
profit              460
dtype: int64